# ScopeSim + IRDB validation for MICADO

This is referencing Ric Davies' document *MICADO Imaging and Spectroscopy Reference*, Doc. No.: ELT-TRE-MCD-56300-0173, Issue: 0.5, Date: 14.06.2023. This document is hereinafter referred to as "the reference document". Any mention of sections, tables and figures refer to those listed in that document.

In [ ]:
import scopesim as sim
from matplotlib import pyplot as plt

# Set local path
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../"

### General info about the optical train object

Note e.g. the `!INST.plate_scale`, which matches with the final plate scale for spectroscopy mode given in **Table 3**.

In [ ]:
cmds = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", "SPEC_3000x16"])
cmds.cmds["!OBS.filter_name_fw1"] = "Spec_IJ"
cmds.cmds["!OBS.filter_name_fw2"] = "open"
micado = sim.OpticalTrain(cmds)

micado

### Specific info about the included effects

Using the `.show_in_notebook()` method to get a non-truncated table.

In [ ]:
micado.effects.show_in_notebook()

## 4.1 Detector and Pixel Numbering

### Figure 3

This is a recreation of the spectral traces on the detector layout, using the actual `OpticalTrain` object in ScopeSim.
Note the orientation is flipped compared to the reference document. Some orders (8_3, 5_2) are missing, as they are missing from the IRDB.

In [ ]:
# Setup figure
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

modes = ["SPEC_15000x20", "SPEC_15000x20", "SPEC_3000x16"]
filters = ["Spec_HK", "J", "Spec_IJ"]

for mode, fltr, ax in zip(modes, filters, axes):
    cmds = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", mode])
    cmds.cmds["!OBS.filter_name_fw1"] = fltr
    cmds.cmds["!OBS.filter_name_fw2"] = "open"
    micado = sim.OpticalTrain(cmds)

    tbl = micado["filter_wheel_1"].current_filter.table
    wav_min = tbl[tbl["transmission"] > 0.001]["wavelength"].min()
    wav_max = tbl[tbl["transmission"] > 0.0]["wavelength"].max()
    xi_min = micado["spectroscopic_slit_aperture"].table["x"].min()
    xi_max = micado["spectroscopic_slit_aperture"].table["x"].max()

    micado["full_detector_array"].plot(axes=ax)
    micado["micado_spectral_traces"].plot(wave_min=wav_min, wave_max=wav_max, xi_min=xi_min, xi_max=xi_max, axes=ax,
                                      plot_footprint=False, plot_wave=False, plot_ctrlpnts=False, plot_outline=True,
                                      plot_trace_id=True)
    ax.set_title(f"{fltr=}, {mode=}")
fig.tight_layout()

As can be seen in the leftmost panel of the plot, order 3_1 has a weird shape towards the bottom. Let's plot the individual control points to see what's going on:

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

mode = "SPEC_15000x20"
fltr = "Spec_HK"

cmds = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", mode])
cmds.cmds["!OBS.filter_name_fw1"] = fltr
cmds.cmds["!OBS.filter_name_fw2"] = "open"
micado = sim.OpticalTrain(cmds)

tbl = micado["filter_wheel_1"].current_filter.table
wav_min = tbl[tbl["transmission"] > 0.001]["wavelength"].min()
wav_max = tbl[tbl["transmission"] > 0.0]["wavelength"].max()
xi_min = micado["spectroscopic_slit_aperture"].table["x"].min()
xi_max = micado["spectroscopic_slit_aperture"].table["x"].max()

micado["full_detector_array"].plot(axes=ax)
micado["micado_spectral_traces"].plot(wave_min=wav_min, wave_max=wav_max, xi_min=xi_min, xi_max=xi_max, axes=ax,
                                  plot_footprint=False, plot_wave=False, plot_ctrlpnts=True, plot_outline=False,
                                  plot_trace_id=False)
ax.set_title(f"{fltr=}, {mode=}")

There seems to be a single erroneous control point with an x-value of 0.0, considering all other points the trace is in line with the reference document.

## 4.2 Detector properties

### Table 4

The positions and angles of the individual detectors. Table is transposed compared to the one in the reference document.

In [ ]:
micado["full_detector_array"].table[["id", "x_cen", "y_cen", "angle"]]

### QE curve

The following plot shows the detector QE, which is related to the information in **Table 5**.

In [ ]:
micado["qe_curve"].plot().show()

## 7.2 Grating Efficiencies
### Figure 10

In [ ]:
micado["grating_efficiency"].plot()

# Limiting magnitude

The following series of images each show a grid of 7x7 stars with magnitudes ranging from 25 to 30 (Vega mags), observed in MCAO mode with the wide-field imaging mode of 4mas/pix, in Ks, H and J filters. The second image (with the colourful boxes) in each filter shows an estimate of signal-to-noise ratios (SNR) for each observed star. The final plots each show a fit through the magnitude and SNR values, with a limiting magnitude at SNR=5 plotted into the data.

In [ ]:
import numpy as np

import scopesim as sim
from scopesim import rc
from scopesim.source import source_templates as st

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from scipy.stats import linregress

import hmbp
from astropy import units as u

args = [("open", "Ks", 2, 27.1, 0.3),
        ("open", "H", 2, 27.5, 0.3),
        ("J", "open", 1, 27.9, 1.0)]
ao_mode="MCAO"

In [ ]:
def lim_mag(fw1, fw2, r0, rics_lim_mag, abslim):
    n_stars, mmin, mmax = 49, 25, 30
    r1, r2 = 10, 15              # aperture radii  r0 (sig), r1-r2 (noise)
    src = st.star_field(n_stars, mmin, mmax, width=3, use_grid=True)

    cmds = sim.UserCommands(use_instrument="MICADO",
                            set_modes=[ao_mode, "IMG_4mas"])
    micado = sim.OpticalTrain(cmds)
    micado.cmds["!OBS.dit"] = 18000
    micado["filter_wheel_1"].change_filter(fw1)
    micado["filter_wheel_2"].change_filter(fw2)
    micado["detector_linearity"].include = False

    micado.observe(src)
    hdul = micado.readout()[0]
    det = hdul[1].data
    imp = micado.image_planes[0].hdu.data  # e-/pixel/s

    fig = plt.figure(figsize=(15, 20))
    ax1, ax2 = fig.subplots(2, 1)
    ax1.imshow(imp, norm=LogNorm(vmin=np.median(imp), vmax=1.01*np.median(imp)))
    ax2.imshow(det, norm=LogNorm(vmin=np.median(det), vmax=1.01*np.median(det)))

    offset = 2      # this needs to be addressed
    xpix, ypix = src.fields[0]["x"].data, src.fields[0]["y"].data
    xpix = xpix / 0.004 + 512 + offset
    ypix = ypix / 0.004 + 512 + offset
    mags = np.round(np.linspace(mmin, mmax, n_stars), 1)

    snrs = []
    for x, y, mag in zip(xpix, ypix, mags):
        x, y = int(x+0.5), int(y+0.5)
        sig_im = np.copy(det[y-r0:y+r0+1, x-r0:x+r0+1])
        bg_im  = np.copy(det[y-r2:y+r2+1, x-r2:x+r2+1])
        bg_im[r1:-r1, r1:-r1] = 0

        bg_median = np.median(bg_im[bg_im > 0])
        bg_std = np.std(bg_im[bg_im > 0])
        # bg_std = np.sqrt(bg_median)
        noise = bg_std * np.sqrt(np.prod(sig_im.shape)) * np.sqrt(2)        # sqrt(2) comes from BG subtraction (see Rics doc)
        signal = np.sum(sig_im - bg_median)
        snr = signal/noise
        snrs += [snr]

        plt.plot([x-r0, x+r0, x+r0, x-r0, x-r0],
                 [y-r0, y-r0, y+r0, y+r0, y-r0], "g")
        plt.plot([x-r1, x+r1, x+r1, x-r1, x-r1],
                 [y-r1, y-r1, y+r1, y+r1, y-r1], "y")
        plt.plot([x-r2, x+r2, x+r2, x-r2, x-r2],
                 [y-r2, y-r2, y+r2, y+r2, y-r2], "r")

        plt.text(x+r2, y+r2, str(round(snr, 1)), color="w")
        plt.text(x-r2, y-r2, str(mag), color="w")
    fltr = fw1 if fw1 != "open" else fw2
    plt.title(f"Filter: {fltr}")
    plt.show()
    return mags, snrs

In [ ]:
def mag_fit(mags, snrs, *args):
    fltr = args[0] if args[0] != "open" else args[1]
    snrs = np.array(snrs)
    results = linregress(mags[snrs > 5], np.log10(snrs[snrs > 5]))
    m, c = results[:2]
    sigma = 5
    lim_mag = (np.log10(sigma) - c) / m

    plt.plot(mags, snrs, ".", alpha=0.3)
    plt.plot(mags, 10 ** (m * mags + c), "r")

    plt.axhline(sigma)
    plt.axvline(lim_mag)
    abmag = hmbp.convert(from_quantity=lim_mag*u.mag, to_unit=u.ABmag, filter_name=fltr)
    txt = f"{lim_mag:.2f} Vega mag\n{abmag:.2f}"
    plt.text(lim_mag, sigma, txt,
             horizontalalignment="left",
             verticalalignment="bottom")
    plt.xlabel("Magnitude")
    plt.ylabel("SNR")
    plt.title(f"Filter: {fltr}")
    plt.show()

In [ ]:
mags, snrs = lim_mag(*args[0])

In [ ]:
mag_fit(mags, snrs, *args[0])

In [ ]:
mags, snrs = lim_mag(*args[1])

In [ ]:
mag_fit(mags, snrs, *args[1])

In [ ]:
mags, snrs = lim_mag(*args[2])

In [ ]:
mag_fit(mags, snrs, *args[2])